In [56]:
# Importing necessary libraries
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
import os
import random

# Libraries for TensorFlow
from tensorflow.keras.utils import to_categorical, plot_model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
from tensorflow import keras

# Library for Transfer Learning
from tensorflow.keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
# Setting the image size for resizing
image_size = 32


def read_image(class_path, image_file):
    """
    Read and preprocess an image from a specified path.

    Parameters:
    - class_path: Path to the directory containing the image
    - image_file: Filename of the image

    Returns:
    - image: Preprocessed and resized grayscale image (2D numpy array)
    """
    image_path = os.path.join(class_path, image_file)
    image =  cv2.imread(image_path)
    image = cv2.resize(image, (image_size, image_size))
    return image


# Creating empty lists to store class labels and images
class_labels_list = []
images_list = []

# Path to the dataset directory
data_directory = './dataset/Validation/'

# List of class directories in the dataset
# directories = os.listdir(data_directory)
directories = ['A','B','C','D','E','F','G']


# Loop through each class directory
for class_label, class_name in enumerate(directories):
    class_path = os.path.join(data_directory, class_name)
    
    # Loop through each image file in the class directory
    for image_file in os.listdir(class_path):
        # Check if the file is a JPEG image
        if image_file.endswith('.jpg'):
            # Read and preprocess the image
            image = read_image(class_path, image_file)
            
            # Append the image and its corresponding class label to the lists
            images_list.append(image)
            class_labels_list.append(class_label)

# Separate data into feature vectors (X) and class labels (y)

data_x = images_list
data_y = class_labels_list


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, 
                                                    test_size=0.25, 
                                                    random_state=13)


In [60]:
datagen = ImageDataGenerator(
        rescale=1./255.,
        zoom_range=0.1,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip = True
    )
aug_data = datagen.flow_from_directory(
        data_directory,
        batch_size=32,
        seed=42,
        class_mode='binary',
        target_size=(image_size, image_size),
        classes={'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6}
    )




# Initialize lists to store generated images and labels
generated_images_list = []
generated_labels_list = []

# Loop through the generator and store the generated images and labels
for batch, labels in datagen.flow(X_train, y_train, batch_size=32):
    # Append the generated images and labels to the lists
    generated_images_list.append(batch)
    generated_labels_list.append(labels)

    # Break the loop after generating the desired number of batches
    if len(generated_images_list) >= 32:
        break

# Concatenate the generated images and labels
generated_images = np.concatenate(generated_images_list, axis=0)
generated_labels = np.concatenate(generated_labels_list, axis=0)

# Print the shape of the generated images and labels
print(f'Shape of generated images: {generated_images.shape}')
print(f'Shape of generated labels: {generated_labels.shape}')


# Concatenate the data
concatenated_data = np.concatenate((X_train, generated_images), axis=0)

# Concatenate the labels of the original data and augmented data
original_labels = y_train
concatenated_labels = np.concatenate((original_labels, generated_labels), axis=0)

x_train_flatten, x_test_flatten = train_test_split(concatenated_data, test_size=0.2, random_state=42)
print(concatenated_data.shape )
print(concatenated_labels.shape)


Found 2262 images belonging to 7 classes.


ValueError: `x` (images tensor) and `y` (labels) should have the same length. Found: x.shape = (32, 32, 3), y.shape = (1696,)

In [ ]:

#define VGG16 Model
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

tf.keras.backend.clear_session()
input_shape = (image_size, image_size, 3)
base_model = tf.keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=input_shape
)
base_model.trainable = False #change to true and comment the difference

model_vgg16 = tf.keras.Sequential()
model_vgg16.add(base_model)
model_vgg16.add(tf.keras.layers.GlobalAveragePooling2D())

# model_vgg16.add(tf.keras.layers.Flatten())
model_vgg16.add(tf.keras.layers.Dense(512, activation='relu'))
model_vgg16.add(tf.keras.layers.Dense(256, activation='relu'))
model_vgg16.add(tf.keras.layers.Dropout(0.5))
model_vgg16.add(tf.keras.layers.Dense(128, activation='relu'))



model_vgg16.add(tf.keras.layers.Dense(85, activation='relu'))
model_vgg16.add(tf.keras.layers.Dense(65, activation='relu'))
model_vgg16.add(tf.keras.layers.Dense(45, activation='relu'))
model_vgg16.add(tf.keras.layers.Dense(39, activation='softmax'))

model_vgg16.compile(loss='SparseCategoricalCrossentropy',
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=['acc'])
model_vgg16.summary()
model_vgg16.save("model.h5")



        


        




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                        

In [ ]:
from keras.callbacks import EarlyStopping
checkpoint = tf.keras.callbacks.ModelCheckpoint(
             filepath='./model.h5',
             monitor='val_accuracy',
             mode='max',
            save_best_only=True,
            verbose=1)
early =EarlyStopping(monitor='val_loss', mode='max',restore_best_weights=True, verbose=1, patience=1)

callbacks_list = [checkpoint]

history = model_vgg16.fit(
        np.array(X_train),
        np.array(y_train),
        validation_data = (np.array(X_test), np.array(y_test)),
        epochs=10,
        batch_size=32,
        verbose=1,
        callbacks=callbacks_list
)

Epoch 1/10
53/53 [==============================] - 6s 100ms/step - loss: 4.7958 - acc: 0.1645 - val_loss: 3.1489 - val_acc: 0.1025
Epoch 2/10
53/53 [==============================] - 5s 102ms/step - loss: 3.0351 - acc: 0.2854 - val_loss: 2.4561 - val_acc: 0.1431
Epoch 3/10
53/53 [==============================] - 5s 103ms/step - loss: 2.3395 - acc: 0.2919 - val_loss: 1.7728 - val_acc: 0.2138
Epoch 4/10
53/53 [==============================] - 6s 116ms/step - loss: 1.7823 - acc: 0.2052 - val_loss: 1.4209 - val_acc: 0.1219
Epoch 5/10
53/53 [==============================] - 7s 136ms/step - loss: 1.5347 - acc: 0.1527 - val_loss: 1.2257 - val_acc: 0.0565
Epoch 6/10
53/53 [==============================] - 7s 138ms/step - loss: 1.3466 - acc: 0.1474 - val_loss: 1.0973 - val_acc: 0.1184
Epoch 7/10
53/53 [==============================] - 6s 121ms/step - loss: 1.2341 - acc: 0.1521 - val_loss: 0.9891 - val_acc: 0.0901
Epoch 8/10
53/53 [==============================] - 6s 120ms/step - loss: 1.